In [1]:
# !pip install gurobipy

In [2]:
from gurobipy import *

 ### ρの値の検討

In [34]:
def order(a,b):
  return [a,b]

In [35]:
O1=order(1,2)
O2=order(2,1)
O=[O1,O1,O1,O1,O2]

S=[i for i in range(1,3)]
H=[i for i in range(1,3)]
P=[i for i in range(1,3)]
T=[i for i in range(1,7)]
n_=[1,1]
n1=[n_[O1[0]-1],n_[O1[1]-1]]

S,H,P,T,n1

([1, 2], [1, 2], [1, 2], [1, 2, 3, 4, 5, 6], [1, 1])

In [36]:
m=Model("IP3")

# 変数の定義
x,y,pre,penalty,minus={},{},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for p in P:
      for i,o in enumerate(O):
        penalty[s,h,i]=m.addVar(vtype="c",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

# 同じ場所でpenaltyが加算される場合がある。
for s in S:
  for h in range(2,len(H)+1):

    for i,o in enumerate(O):
      # m.addConstr(minus[s,h,i]<=quicksum(penalty[s,h,p,i] for p in P))
      for j,p in enumerate(o):
        m.addConstr(penalty[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

In [37]:
m.addConstr(x[1,1,O1[1],1]==1)
m.addConstr(x[1,2,O1[0],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [38]:
m.setObjective(quicksum(y[s,k,t] for s in S for k in S if k!=s for t in range(1,len(T)))
+1/len(O)*quicksum(penalty[s,h,i] for s in S for h in H if h!=1 for i in range(len(O))))

In [39]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 171 rows, 78 columns and 552 nonzeros
Model fingerprint: 0x46630fe6
Variable types: 20 continuous, 58 integer (58 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 171 rows and 78 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 0.2 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e-01, best bound 2.000000000000e-01, gap 0.0000%
2


In [40]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    # if t==len(T):
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in penalty:
    if penalty[s,h,i].X>EPS:
      print("penalty[%2s,%2s,%2s]=%3s" %(s,h,i,penalty[s,h,i].X))
  print("the objective function", m.objVal)

x[ 1, 1, 2, 1]=1.0
x[ 1, 2, 1, 1]=1.0
penalty[ 1, 2, 4]=1.0
the objective function 0.2


## 問題例11 複数のシナリオを生成し同時に解く

In [65]:
def order(a,b,c,d):
  return [a,b,c,d]

In [66]:
# nと初期配置を変更しなければいけない
O1=order(1,2,3,4)
O2=order(3,1,2,4)
O3=order(3,4,1,2)
O4=order(1,2,4,3)
O5=order(2,3,1,4)
O6=order(2,1,3,4)
O7=order(1,3,2,4)

O=[O1,O1,O1,O1,O1,O2,O3,O3,O3,O4,O4,O4,O5]
O_=[O1,O6]
S=[i for i in range(1,5)]
H=[i for i in range(1,5)]
P=[i for i in range(1,5)]
T=[i for i in range(1,7)]
n_=[1,2,4,3]
n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

S,H,P,T,n1

([1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4, 5, 6], [1, 2, 4, 3])

In [67]:
m=Model("IP3")

# 変数の定義
x,y,pre,penalty,minus={},{},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for p in P:
      for i,o in enumerate(O):
        penalty[s,h,i]=m.addVar(vtype="c",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

# 同じ場所でpenaltyが加算される場合がある。
for s in S:
  for h in range(2,len(H)+1):

    for i,o in enumerate(O):
      # m.addConstr(minus[s,h,i]<=quicksum(penalty[s,h,p,i] for p in P))
      for j,p in enumerate(o):
        m.addConstr(penalty[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

In [68]:
# 初期配置の制約

m.addConstr(x[1,1,O1[3],1]==1)
m.addConstr(x[1,2,O1[3],1]==1)
m.addConstr(x[1,3,O1[2],1]==1)
m.addConstr(x[2,1,O1[3],1]==1)
m.addConstr(x[2,2,O1[2],1]==1)
m.addConstr(x[2,3,O1[1],1]==1)
m.addConstr(x[3,1,O1[2],1]==1)
m.addConstr(x[3,2,O1[2],1]==1)
m.addConstr(x[4,1,O1[1],1]==1)
m.addConstr(x[4,2,O1[0],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [69]:
m.setObjective(quicksum(y[s,k,t] for s in S for k in S if k!=s for t in range(1,len(T)))
+1/(len(O)**0.25)*quicksum(penalty[s,h,i] for s in S for h in H if h!=1 for i in range(len(O))))

In [70]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1643 rows, 1068 columns and 9590 nonzeros
Model fingerprint: 0x6791b857
Variable types: 624 continuous, 444 integer (444 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 823 rows and 711 columns
Presolve time: 0.13s
Presolved: 820 rows, 357 columns, 4471 nonzeros
Variable types: 0 continuous, 357 integer (297 binary)
Found heuristic solution: objective 17.2192905

Root relaxation: objective 1.992994e+00, 571 iterations, 0.04 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.99299    0  116   17.21929    1.99299  88.4%     -    0s
H    0     0                 

In [71]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    # if t==len(T):
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in penalty:
    if penalty[s,h,i].X>EPS:
      print("penalty[%2s,%2s,%2s]=%3s" %(s,h,i,penalty[s,h,i].X))
  print("the objective function", m.objVal)

x[ 1, 1, 4, 1]=1.0
x[ 1, 2, 4, 1]=1.0
x[ 1, 3, 2, 3]=1.0
x[ 1, 3, 3, 1]=1.0
x[ 2, 1, 4, 1]=1.0
x[ 2, 2, 3, 1]=1.0
x[ 2, 3, 2, 1]=1.0
x[ 3, 1, 3, 1]=1.0
x[ 3, 2, 3, 1]=1.0
x[ 3, 3, 3, 2]=1.0
x[ 3, 4, 3, 4]=1.0
x[ 4, 1, 2, 1]=1.0
x[ 4, 2, 1, 1]=1.0
y[ 1, 3, 1]=1.0
y[ 2, 1, 2]=1.0
y[ 2, 3, 3]=1.0
penalty[ 1, 3, 6]=1.0
penalty[ 1, 3, 7]=1.0
penalty[ 1, 3, 8]=1.0
penalty[ 4, 2,12]=1.0
the objective function 5.106561551391707


In [64]:
len(O)

13

In [42]:
O

[[1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [3, 1, 2, 4],
 [3, 4, 1, 2],
 [3, 4, 1, 2],
 [3, 4, 1, 2],
 [1, 2, 4, 3],
 [1, 2, 4, 3],
 [1, 2, 4, 3],
 [2, 3, 1, 4]]